# Pull data and github repo

In [1]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PROJECT_PATH = "/content/drive/MyDrive/animal-classification"
MODEL_FOLDER = "models"
LOG_FOLDER = "logs"
NOTEBOOK_FILE = "animal-classification.ipynb"
GITHUB_PROJECT_PATH = "/content/animals-10"
DATA_PATH = "/content/dataset/animals10/raw-img"

Mounted at /content/drive


In [3]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'animals10:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F59760%2F840806%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240530%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240530T065741Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D55fed7b0a79aee07278d3bf8ee07e1bcc76a8fac79e9dc88d25bc4a73ecbf9ebca7fd16d9a9982b896c99969c7f0132c9e5eb6f50732c183382c38ba40aad851c55b8e736ea047d41eed11e5b4b55ffd6e782b789c45f2b427dac29beead372ff3bfcf097509a2118e10d6d507f2a87d6be0443166b89f61a7ddda6282c032bb23fcae408744a8d25da68df60a12dcc314606e522f6ddf0ba753bb20da9019eac5145a8a2266fe878731d44a45fbe6b8fb601fd796a386ce3bf594578caed467825df6fae8a9d5f76aa0215b9d36b23bc4ec4167962488c1f3d5b10f389d9deee3efd3ddb90594ea7626877042348f1b51b336307db9c2946742da4c35010d35'

KAGGLE_INPUT_PATH='dataset'

!umount /kaggle/input/ 2> /dev/null
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')



[==================================================] 614087302 bytes downloaded
Downloaded and uncompressed: animals10
Data source import complete.


Pull from github and switch branch

In [ ]:
!chmod +x config.sh
!./config.sh

In [ ]:
%cd animals-10

In [ ]:
!git pull origin modelling

In [ ]:
!git branch modelling
!git checkout modelling

Push changes to github

In [ ]:
!rm animal_classification.ipynb
!cp "/content/drive/MyDrive/animal-classification/animals10" .
!mv animal_classification animals10.ipynb

In [ ]:
!rm .gitignore
!touch .gitignore
!echo "__pycache__/" >> .gitignore
!echo "logs/" >> .gitignore
!echo "models/*.keras" >> .gitignore

In [ ]:
!git add *
!git commit -m 'Fix identation'  # commit in Colab
!git push origin modelling